(case_kvlcc2)=
## KVLCC2

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
plt.style.use('paper')
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf')

from src.symbols import *

# Read configs:
conf_path = os.path.join("../../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmm_names = global_variables["vmms"]

from wPCC_pipeline.pipelines.motion_regression.nodes import predict_force, fit_motions, create_model_from_motion_regression
from wPCC_pipeline.pipelines.prediction.nodes import simulate_euler
from src.prime_system import PrimeSystem

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.pipeline import Pipeline
from src.parameters import df_parameters
p = df_parameters['symbol']
from src.feature_selection import drop_multicollinearity
from src.bias_variance_tradeoff import train_test_split_run
from sklearn.metrics import r2_score
from wPCC_pipeline.pipelines.motion_regression.nodes import predict_force, fit_motions, create_model_from_motion_regression, create_full_model_from_motion_regression
from src.models.regression import Regression
from myst_nb import glue
import src.symbols as s
from IPython.display import Latex
from src.substitute_dynamic_symbols import run
import seaborn as sns
from src.feature_selection import feature_imporance, BestFeatures, DropCorrelation
from wPCC_pipeline.turning_circle import TurningCircle
from src.models.propeller import fit, predict, predictor
from jb_helpers import parameter_to_latex, df_to_myst
from wPCC_pipeline.monte_carlo import monte_carlo, parameter_variation

In [ ]:
ship="kvlcc2_hsva"
#vmm_name = "vmm_martins_simple"
vmm_name = "vmm_abkowitz"
vmm = catalog.load(vmm_name)

ship_data = catalog.load(f"{ship}.ship_data")

#regression = catalog.load(f"{ship}.updated.{vmm_name}.joined.regression")
#regression.diff_eq_Y.exclude_parameters.pop('Ydelta')
#regression.diff_eq_Y.exclude_parameters.pop('Ythrustdelta')


data = catalog.load(f"{ship}.updated.joined.data_ek_smooth")

added_masses = catalog.load(f"{ship}.added_masses")
exclude_parameters = catalog.load(f"params:{ship}.motion_regression.exclude_parameters")
columns = ['u','v','r','u1d','v1d','r1d','delta','thrust','id','x0','y0','psi','rev']
data_with_force = predict_force(data=data[columns], added_masses=added_masses, ship_parameters=ship_data, vmm=vmm)

In [ ]:
def break_plot(df, **kwargs):
    
    df_ = df.copy()
    mask = np.concatenate([[False],np.diff(df_.index)>1])
    df_.loc[mask] = np.NaN
    df_.plot(**kwargs)


In [ ]:
ids = list(data_with_force['id'].unique())
ids_train = ids.copy()
# id_test = 22774  (MARIN test as test)
#ids_train.remove(id_test)
mask = data_with_force['id'].isin(ids_train)
data_train = data_with_force.loc[mask].copy()
#assert not id_test in data_train['id'].unique()

ps = PrimeSystem(**ship_data)
data_prime = ps.prime(data_train, U=data_with_force['U'])

In [ ]:
ids_test = [
    'HSVA_CPMC_KVLCC2_Z_35_05',
]

mask = data_train['id'].isin(ids_test)
data_validation = data_train.loc[mask].copy()
data_sub_train = data_train.loc[~mask].copy()

In [ ]:
ids2 = ["kvlcc2.updated.MARIN_FREE_KVLCC2_tc_35_m",
        "kvlcc2.updated.MARIN_FREE_KVLCC2_tc_-35_m",
        "kvlcc2.updated.MARIN_FREE_KVLCC2_zz_10_m",
        "kvlcc2.updated.MARIN_FREE_KVLCC2_zz_-10_m",
        "kvlcc2.updated.MARIN_FREE_KVLCC2_zz_20_m",
        "kvlcc2.updated.MARIN_FREE_KVLCC2_zz_-20_m",
        
       ]

data_testing = pd.DataFrame()
for id in ids2: 
    df_test = catalog.load(f"{id}.data_ek_smooth")
    df_test['id'] = id
    data_testing = data_testing.append(df_test)
    

In [ ]:
def rotate(df):
    
    dpsi = 2*np.pi*np.random.random()
    
    df['psi']+=dpsi
    x0 = df['x0'].copy()
    y0 = df['y0'].copy()
    
    df['x0'] = np.cos(dpsi)*x0 - np.sin(dpsi)*y0
    df['y0'] = np.sin(dpsi)*x0 + np.cos(dpsi)*y0
        
    
    return df
    
    

In [ ]:
np.random.seed(1)
fig,ax=plt.subplots()

dataframes = {id:rotate(df_) for id, df_ in data_sub_train.groupby(by='id')}
styles = {id:{'style':'b-','label':'_nolegend_'} for id, df_ in data_sub_train.groupby(by='id')}

dataframes.update({id:rotate(df_) for id, df_ in data_validation.groupby(by='id')})
styles.update({id:{'style':'g-','label':'_nolegend_'} for id, df_ in data_validation.groupby(by='id')})

dataframes.update({id:rotate(df_) for id, df_ in data_testing.groupby(by='id') if 'tc' in id})
styles.update({id:{'style':'r-','label':'_nolegend_'} for id, df_ in data_testing.groupby(by='id')})

track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'],  styles=styles, N=2, ax=ax);

ax.plot([],'b-',label='Training (HSVA)')
ax.plot([],'g-',label='Validation (HSVA)')
ax.plot([],'r-',label='Testing (MARIN)')


ax.legend();
glue('fig_kvlcc2_traintest',fig, display=False)

In [ ]:
ps = PrimeSystem(**ship_data)
regression = Regression(
        vmm=vmm,
        data=data_train,
        added_masses=added_masses,
        ship_parameters=ship_data,
        prime_system=ps,
        exclude_parameters=exclude_parameters, 
        connect_equations_Y_N_rudder=True
    )

In [ ]:
Xs = {}
ys = {}


Xs['X'], ys['X'] = regression.diff_eq_X.calculate_features_and_label(data=data_prime, y=data_prime['fx'])
Xs['Y'], ys['Y'] = regression.diff_eq_Y.calculate_features_and_label(data=data_prime, y=data_prime['fy'])
Xs['N'], ys['N'] = regression.diff_eq_N.calculate_features_and_label(data=data_prime, y=data_prime['mz'])

dofs = list(Xs.keys())

In [ ]:
df_parameters = pd.DataFrame()
df_parameters['mean'] = regression.model_N.params
df_parameters['std'] = regression.model_N.bse
df_parameters.sort_values(by='std', ascending=False, inplace=True)

In [ ]:
df_table = df_parameters.iloc[0:5]
glue('Ndelta',df_parameters.loc['Ndelta','mean'])
glue('eNdelta',df_parameters.loc['Ndelta','std'])

glue('Nvvdelta',df_parameters.loc['Nvvdelta','mean'])
glue('eNvvdelta',df_parameters.loc['Nvvdelta','std'])

df_table

In [ ]:
ship_model = catalog.load(f"{ship}.updated.{vmm_name}.joined.model")

In [ ]:
propeller_coefficients = catalog.load("kvlcc2.propeller_coefficients")

for key,value in ship_data.items():
    glue(key,value, display=False)
    print(key)

for key,value in propeller_coefficients.items():
    glue(key,np.round(value, 5), display=False)
    print(key)

### KVLCC2 propeller model

The coefficients of $K_T$ [eq](eqkt) where regressed from the propeller KVLCC2 characteristics from SIMMAN2008 HSVA model tests {cite:p}`stern_experience_2011` ($k_0$:{glue:}`k_0`, $k_1$:{glue:}`k_1`, $k_2$:{glue:}`k_2`).

The Polynomial propeller model was developed with polynomial regression and cross validation on the training and validation datasets, to make the best feature selection.

A cross validation study on the three candidate propeller models: MMG propeller model, Simple propeller model and the Polynomial propeller model where carried out. The training set as well as the validation set was made up by entire model test time series from the HSVA model tests.
The model tests were divided into the test and validation set randomly. The random train and validation was reapeated 100 times. The Polynomial was selected, having the highest accuracy. Taylor wake $w_{p0}$ = {glue:}`w_p0` was used in all three models, the MMG model used $C_1$={glue:}`C_1`, $C_2$={glue:}`C_2_beta_p_pos` when $\beta_p>0$ and $C_2$={glue:}`C_2_beta_p_neg` when $\beta_p<=0$ {cite:p}`yasukawa_introduction_2015-1`. 

{numref}`fig_propeller_validation` shows a small part of the cross validation.

```{figure} figures/fig_propeller_validation.pdf
---
name: fig_propeller_validation
---
Validation of MMG, Simple and Poynomial propeller models for KVLCC2.
```


{numref}`KVLCC2_propeller_model` shows coefficients fitted on the training and validation dataset for KVLCC2.

```{list-table} KVLCC2 propeller model
:header-rows: 1
:name: KVLCC2_propeller_model
* -  
  - $\beta_p>0$
  - $\beta_p<=0$
* - $C_1$
  - -0.1735
  - -0.1066
* - $C_2$
  - 0.4589
  - 0.0771
* - $C_3$
  - -1.8865
  - 1.2958
* - $C_4$
  - 0.0515
  - 0.0514

```

### KVLCC2 VMM

In [ ]:
vmm_martin = catalog.load("vmm_martins_simple")
ek_martin = catalog.load(f"{ship}.vmm_martins_simple.ek")

In [ ]:
#regression_martin, _ = fit_motions(data=data_sub_train, 
#                                   added_masses=added_masses, 
#                                   ship_data=ship_data, 
#                                   vmm=vmm_martin, 
#                                   exclude_parameters=exclude_parameters)
#
#ship_model_martin = create_model_from_motion_regression(regression=regression_martin)


In [ ]:
add_constant = False
model_pos, model_neg = fit(data=data_sub_train, ship_data=ship_data, propeller_coefficients=propeller_coefficients, add_constant=add_constant) 
data_predicted_thrust = predict(model_pos=model_pos, model_neg=model_neg, data=data_sub_train, propeller_coefficients=propeller_coefficients, 
                                ship_data=ship_data)

regression_martin, parameters = fit_motions(data=data_predicted_thrust, added_masses=added_masses, 
                                                      ship_data=ship_data, vmm=vmm_martin, exclude_parameters=exclude_parameters)

ship_model_martin = create_full_model_from_motion_regression(regression=regression_martin, 
                                                                  model_pos=model_pos, 
                                                                  model_neg=model_neg, 
                                                                  propeller_coefficients=propeller_coefficients)

In [ ]:
#regression_abkowitz, _ = fit_motions(data=data_sub_train, 
#                                   added_masses=added_masses, 
#                                   ship_data=ship_data, 
#                                   vmm=vmm, 
#                                   exclude_parameters=exclude_parameters)
#
#ship_model_abkowitz = create_model_from_motion_regression(regression=regression_abkowitz)

In [ ]:
add_constant = False
model_pos, model_neg = fit(data=data_sub_train, ship_data=ship_data, propeller_coefficients=propeller_coefficients, add_constant=add_constant) 
data_predicted_thrust = predict(model_pos=model_pos, model_neg=model_neg, data=data_sub_train, propeller_coefficients=propeller_coefficients, 
                                ship_data=ship_data)

regression_abkowitz, parameters = fit_motions(data=data_predicted_thrust, added_masses=added_masses, 
                                                      ship_data=ship_data, vmm=vmm, exclude_parameters=exclude_parameters)

ship_model_abkowitz = create_full_model_from_motion_regression(regression=regression_abkowitz, 
                                                                  model_pos=model_pos, 
                                                                  model_neg=model_neg, 
                                                                  propeller_coefficients=propeller_coefficients)

Forces and moment for the hull, rudder and propeller predicted with the VMM:s fitted on the training set are show in {numref}`fig_kvlcc2_validation_forces`. 
Simulations of the validation cases where only possible with the Modified Abkowitz model as shown for one of the ZigZag20/20 validation cases in {numref}`fig_kvlcc2_validation_sim`. The predicted thrust is also shown in this figure.

```{glue:figure} fig_kvlcc2_validation_forces
:figwidth: 1000px
:name: "fig_kvlcc2_validation_forces"

Validation of force models for KVLCC2.
```

```{glue:figure} fig_kvlcc2_validation_sim
:figwidth: 1000px
:name: "fig_kvlcc2_validation_sim"

Validation with simulations for KVLCC2.
```

In [ ]:
validation_martin = regression_martin.predict(data_validation)
validation_abkowitz = regression_abkowitz.predict(data_validation)

data_predicted_thrust = predict(model_pos=model_pos, model_neg=model_neg, data=data_validation, propeller_coefficients=propeller_coefficients, 
                                ship_data=ship_data)

validation_martin['thrust'] = data_predicted_thrust['thrust']
validation_abkowitz['thrust'] = data_predicted_thrust['thrust']

fig,axes=plt.subplots(nrows=4)

for ax,dof in zip(axes,['thrust','fx','fy','mz']):
    
    validation_abkowitz.plot(y=dof,ax=ax, label='Validation AVMM', style='r-')
    validation_martin.plot(y=dof,ax=ax, label='Validation MAVMM', style='b--')
    data_validation.plot(y=dof,ax=ax, label='True', style='k-.')
    
    ax.set_ylabel(dof)
    ax.set_xlabel('sample')
    ax.get_legend().set_visible(False)
    
axes[0].legend();
glue("fig_kvlcc2_validation_forces",fig, display=False)

In [ ]:
for id_validation in ids_test[0:1]:
    
    df_test = catalog.load(f'{ship}.updated.{id_validation}.data_ek_smooth')
    result_martin = ship_model_martin.simulate(df_test.drop(columns=['thrust']))   
    
        
    df_result = result_martin.result.copy()
    df_result['thrust'] = predict(model_pos=model_pos, 
                                                   model_neg=model_neg, 
                                                   data=df_result, 
                                                   propeller_coefficients=propeller_coefficients, 
                                                   ship_data=ship_data)['thrust']
    
    dataframes = {'True':df_test,
                 'Validation MAVMM': df_result,
                 }
    
    #try:
    #    result_abkowitz = ship_model_abkowitz.simulate(df_test)
    #except:
    #    pass
    #else:
    #    dataframes['Abkowitz']=result_abkowitz.result
    
    track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'],  N=2);
    fig = plot(dataframes=dataframes, keys=['thrust','u','v','r'], ncols=1, zero_origo=True);

glue("fig_kvlcc2_validation_sim",fig, display=False)

In [ ]:
#df_test = catalog.load(f'{ship}.updated.{id_test}.data_ek_smooth')
ek = catalog.load(f"{ship}.{vmm_name}.ek")

In [ ]:
add_constant = False
model_pos, model_neg = fit(data=data_train, ship_data=ship_data, propeller_coefficients=propeller_coefficients, add_constant=add_constant) 
data_predicted_thrust = predict(model_pos=model_pos, model_neg=model_neg, data=data_train, propeller_coefficients=propeller_coefficients, 
                                ship_data=ship_data)

regression_martin, parameters = fit_motions(data=data_predicted_thrust, added_masses=added_masses, 
                                                      ship_data=ship_data, vmm=vmm_martin, exclude_parameters=exclude_parameters)

ship_model_martin = create_full_model_from_motion_regression(regression=regression_martin, 
                                                                  model_pos=model_pos, 
                                                                  model_neg=model_neg, 
                                                                  propeller_coefficients=propeller_coefficients)

catalog.save(f"{ ship }.updated.vmm_martins_simple.joined.regression",
             regression_martin)

catalog.save(f"{ ship }.propeller_model_pos",
             model_pos)

catalog.save(f"{ ship }.propeller_model_neg",
             model_neg)



In [ ]:
test_prediction = {}

for id, df_test in data_testing.groupby(by='id', sort=False):
    
    result_martin = ship_model_martin.simulate(df_test)   
    
    df_result = result_martin.result.copy()
    df_result['thrust'] = predict(model_pos=model_pos, 
                                                   model_neg=model_neg, 
                                                   data=df_result, 
                                                   propeller_coefficients=propeller_coefficients, 
                                                   ship_data=ship_data)['thrust']
    
    
    test_prediction[id] = df_result
    

In [ ]:
test_true = data_testing.id.unique()
for id in test_true:
    dataframes = {
        'True' : data_testing.groupby(by='id', sort=False).get_group(id),
        'Test MAVMM' : test_prediction[id],
            
    }
    track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'],  styles=styles, N=2);
    fig = plot(dataframes=dataframes, keys=['u','v','r','psi'], ncols=1,);


In [ ]:
test_true = data_testing.id.unique()
id = test_true[0]
df_test = data_testing.groupby(by='id', sort=False).get_group(id)

In [ ]:
df_parameter_variations = parameter_variation(regression = regression_martin, N_=100, seed=42)

interesting = ['u','v','r','x0','y0','psi','delta','rev','U']
data = df_test[interesting].copy()
try:
    loaders = catalog.load("kvlcc2_hsva.monte_carlo_simulation")
except Exception:
    # Slow way...
    dataframes_monte_carlo = monte_carlo(data, df_parameter_variations, model=ship_model_martin, fast=False)
    catalog.save("kvlcc2_hsva.monte_carlo_simulation", data={str(key):value for key,value in dataframes_monte_carlo.items()})
else:
    dataframes_monte_carlo = {key:loader() for key,loader in loaders.items()}

In [ ]:
dataframes = {
    'Model test' : df_test,
    'Test MAVMM' : test_prediction[id],
        
}

styles = {
    'Model test' : {'style':'r--'},
    'Test MAVMM' : {'style':'k-'},
    
}

for index in dataframes_monte_carlo.keys():
    styles[index] = {'style':'b-', 
                     'alpha':0.1,
                     'label':'_Hidden label',
                     'zorder' : -100}

In [ ]:
fig,ax=plt.subplots()
track_plots(dataframes_monte_carlo, lpp=ship_data['L'], beam=ship_data['B'],  styles=styles, plot_boats=False, ax=ax);
track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'],  styles=styles, N=5, ax=ax);
ax.plot([],[],'b-',label='Monte Carlo MAVMM')
ax.legend()

glue("fig_track_plot_testing_sim", fig, display=False)
fig.savefig("figures/fig_track_plot_testing_sim.pdf")

Result from the final prediction of the turning cirlce test is seen in {numref}`fig_kvlcc2_testing_sim`. The prediction is conducted using simulation with the MAVMM trained on the training and validation dataset. Monte Carlo simulations with alternative realizations of the regression are also shown in this figure. The alternative realizations are very similar to the model with mean values of the regression (black line), since the standard deviation of the regressed parameters are very small.   

```{glue:figure} fig_kvlcc2_testing_sim
:figwidth: 1000px
:name: "fig_kvlcc2_testing_sim"

Comparison between predicted Turning circle test with MAVMM trained on HSVA data and MARIN model test results for KVLCC2.
```

In [ ]:
dataframes_ = dataframes.copy()
dataframes_.update(dataframes_monte_carlo)
fig = plot(dataframes=dataframes_, keys=['u','v','r'], ncols=1, styles=styles, zero_origo=False);
ax = fig.axes[0]
ax.plot([],[],'b-',label='Monte Carlo MAVMM')
ax.legend()
glue("fig_kvlcc2_testing_sim",fig, display=False)

In [ ]:
true_turning_circle = pd.DataFrame()
prediction_turning_circle = pd.DataFrame()

for id, df_result in test_prediction.items():
    
    df_test = data_testing.groupby(by='id', sort=False).get_group(id)
    
    if 'tc' in id:
        
        turning_circle = TurningCircle(angle=35, nominal_speed=df_test['U'].iloc[0], lpp=ship_data['L'], df=df_test)
        true_turning_circle[id] = pd.Series(turning_circle.evaluate(), name='True')
        
        df_result['V'] = np.sqrt(df_result['u']**2 + df_result['v']**2)
        turning_circle = TurningCircle(angle=35, nominal_speed=df_test['U'].iloc[0], lpp=ship_data['L'], df=df_result)
        prediction_turning_circle[id] = pd.Series(turning_circle.evaluate(), name='Test Modified Abkowitz')

true_turning_circle.drop(index='units',inplace=True)
prediction_turning_circle.drop(index='units',inplace=True)

In [ ]:
prediction_turning_circle

In [ ]:
df_advance = pd.DataFrame()
df_advance['True'] = true_turning_circle.loc['advance']
df_advance['Prediction'] = prediction_turning_circle.loc['advance']
df_advance['IMO'] = prediction_turning_circle.loc['Advance (IMO)']
df_advance['delta'] = [35,-35]
#glue("tab_advance",df_advance)
df_advance=df_advance.astype(float)

In [ ]:
df_tactical_diameter = pd.DataFrame()
df_tactical_diameter['True'] = true_turning_circle.loc['tactical_diameter']
df_tactical_diameter['Prediction'] = prediction_turning_circle.loc['tactical_diameter']
df_tactical_diameter['IMO'] = prediction_turning_circle.loc['Tactical diameter (IMO)']
df_tactical_diameter['delta'] = [35,-35]
#glue("tactical_diameter",df_tactical_diameter)
df_tactical_diameter=df_tactical_diameter.astype(float)

In [ ]:
df_table = df_advance[['delta']]
df_table.rename({'delta':'rudder angle [deg]'})
df_table['A (model test) [m]'] = df_advance['True']
df_table['A (prediction) [m]'] = df_advance['Prediction']
df_table['A (IMO) [m]'] = df_advance['IMO']

df_table['TD (model test) [m]'] = df_tactical_diameter['True']
df_table['TD (prediction) [m]'] = df_tactical_diameter['Prediction']
df_table['TD (IMO) [m]'] = df_tactical_diameter['IMO']

In [ ]:
def accuracy(df):
    return ((df['True']-df['Prediction'])/df['True']*100.0).abs().astype(float)

accuracy_advance = accuracy(df_advance).mean()
accuracy_tactical_diameter = accuracy(df_tactical_diameter).mean()

glue("kvlcc2_accuracy_advance", int(np.round(accuracy_advance,0)))
glue("kvlcc2_tactical_diameter", int(np.round(accuracy_tactical_diameter,0)))



For KVLCC2 comparisons of turning circle advance and tactical diameters compared to the model test result is shown in {numref}`tab_kvlcc2_advance`. Predicted advance and tactical diameter differes 2% and 5%.

In [ ]:
table_advance = df_advance.round(decimals=2)[['delta','True','Prediction','IMO']]


print(df_to_myst(df_table.round(decimals=2), 
                 title="KVLCC2 Predicted turning circle advance (A) and tactical diameter (TD) compared to MARIN model tests and IMO limit",
                 name="tab_kvlcc2_advance",
                 include_index=False)
                 )

```{list-table} KVLCC2 Predicted turning circle advance (A) and tactical diameter (TD) compared to MARIN model tests and IMO limit
:header-rows: 1
:name: tab_kvlcc2_advance
* - delta
  - A (model test) [m]
  - A (prediction) [m]
  - A (IMO) [m]
  - TD (model test) [m]
  - TD (prediction) [m]
  - TD (IMO) [m]
* - 35.0
  - 21.59
  - 21.21
  - 31.5
  - 21.72
  - 23.07
  - 35.0
* - -35.0
  - 22.54
  - 22.1
  - 31.5
  - 23.55
  - 24.29
  - 35.0

```

In [ ]:
vmm_name = 'vmm_martins_simple'
ship = 'kvlcc2_hsva'
regression = catalog.load(f"{ ship }.updated.{ vmm_name }.joined.regression")

models = {'X': regression.model_X,
          'Y': regression.model_Y,
          'N': regression.model_N}

for dof, model in models.items():

    mean_parameters = model.bse
    names = pd.Series(mean_parameters.index)
    mean_parameters.index = names.apply(parameter_to_latex)
    mean_parameters_table = mean_parameters.copy()
    
    mean_parameters_table*=1000
    mean_parameters_table=np.around(mean_parameters_table, decimals=3)
    
    print(df_to_myst(mean_parameters_table.to_frame().transpose(), 
                     title=f'{dof} equation derivatives (times 1000)', 
                     name=f"kvlcc2_derivatives_{dof}", include_index=False))


```{list-table} X equation derivatives (times 1000)
:header-rows: 1
:name: kvlcc2_derivatives_X
* - $ X_{vr} $
  - $ X_{\delta\delta} $
  - $ X_{uu} $
  - $ X_{rr} $
  - $ X_{u} $
* - 0.272
  - 0.013
  - 0.618
  - 0.068
  - 0.618

```


```{list-table} Y equation derivatives (times 1000)
:header-rows: 1
:name: kvlcc2_derivatives_Y
* - $ Y_{r} $
  - $ Y_{ur} $
  - $ Y_{T} $
  - $ Y_{u} $
  - $ Y_{v} $
* - 0.654
  - 0.658
  - 1.23
  - 0.003
  - 0.02

```


```{list-table} N equation derivatives (times 1000)
:header-rows: 1
:name: kvlcc2_derivatives_N
* - $ N_{\delta} $
  - $ N_{r} $
  - $ N_{T} $
  - $ N_{u} $
  - $ N_{v} $
  - $ N_{T\delta} $
  - $ N_{vv\delta} $
  - $ N_{ur} $
* - 0.003
  - 0.179
  - 0.274
  - 0.001
  - 0.016
  - 0.941
  - 0.089
  - 0.183

```